In [ ]:
import logging

# Our imports
import emission.core.get_database as edb
import emission.analysis.modelling.tour_model.cluster_pipeline as pipeline
import emission.analysis.modelling.tour_model.similarity as similarity
import emission.analysis.modelling.tour_model.featurization as featurization
import emission.analysis.modelling.tour_model.representatives as representatives
import emission.storage.decorations.analysis_timeseries_queries as esda
import pandas as pd
from numpy import *
# import confirmed_trips_eval_bins_clusters as evaluation
from sklearn import metrics

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
participant_uuid_obj = list(edb.get_profile_db().find({"install_group": "participant"}, {"user_id": 1, "_id": 0}))
all_users = [u["user_id"] for u in participant_uuid_obj]

In [ ]:
radius = 300

## Choose one user for experiment

In [ ]:
user = all_users[4]

In [ ]:
#read the data from the database. We choose key='confirmed_trip' to get confirmed trips here
trips = pipeline.read_data(uuid=user,key=esda.CONFIRMED_TRIP_KEY)

In [ ]:
# select trips that have user_input to analyze
non_empty_trips = [t for t in trips if t["data"]["user_input"] != {}]

In [ ]:
len(non_empty_trips), non_empty_trips

In [ ]:
bin_trips, bins = pipeline.remove_noise(non_empty_trips, radius)

In [ ]:
logging.debug('The list of bins is %s' % bins)

In [ ]:
# show all user labels in all bins
for bin in bins:
    bin_user_input = (non_empty_trips[i].data["user_input"] for i in bin)
    bin_df = pd.DataFrame(data = bin_user_input)
    print(bin_df)

In [ ]:
# turn all user_input into list without binning
bin_trips_user_input_ls = pd.DataFrame(data=[bin_trips[i]["data"]["user_input"] for i in range(len(bin_trips))]).values.tolist()
bin_trips_user_input_ls

In [ ]:
for i in bin_trips_user_input_ls:
    if i[2] == "same_mode":
        print(i)

In [ ]:
# drop duplicate user_input
no_dup_df=pd.DataFrame(data=[bin_trips[i]["data"]["user_input"] for i in range(len(bin_trips))]).drop_duplicates()
no_dup_df

In [ ]:
# turn non-duplicate user_input into list
no_dup_list = no_dup_df.values.tolist()
no_dup_list

In [ ]:
# collect labels_true based on user_input
labels_true =[]
for trip in bin_trips_user_input_ls:
    if trip in no_dup_list:
        labels_true.append(no_dup_list.index(trip))
labels_true

In [ ]:
# collect labels_pred based on bins
labels_pred = []
for i in range(len(bins)):
    for trip in bins[i]:
        labels_pred.append(i)
labels_pred

In [ ]:
metrics.homogeneity_score(labels_true, labels_pred)

In [ ]:
metrics.completeness_score(labels_true, labels_pred)

In [ ]:
metrics.v_measure_score(labels_true, labels_pred)